In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed) 
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "BiasedMNIST",
            'fairness_agg': 'mean',
            'model': 'resnet18',

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 50,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 128,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 5,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha': 0.002,
            'lambda': 1.0,
            'lambda_old': 0.0,
              }
    


#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_tau={params['tau']}_alpha={params['alpha']}"
    if params['lambda'] != 0:
        trial_id+=f"_lmbd_{params['lambda']}_lmbdold_{params['lambda_old']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=BiasedMNIST/seed=10_epoch=50_lr=0.001_tau=5_alpha=0.002_lmbd_1.0_lmbdold_0.0


In [2]:
"MNIST" in params['dataset']

True

In [3]:
from datasets import BiasedMNIST

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                random_class_idx=True)
    input_dim = (3, 28, 28)
    class_idx = benchmark.class_idx
    num_classes = len(class_idx)

[8 2 5 6 3 1 0 7 4 9]


In [4]:
from torchvision.transforms.functional import to_pil_image
from cl_gym.benchmarks.transforms import MNIST_MEAN, MNIST_STD
COLOR_MAP = {
    0: (1, 0, 0),
    1: (0, 1, 0),
    2: (1, 1, 0),
    3: (0, 0, 1),
    4: (1, 0.65, 0),
    5: (0.5, 0, 0.5),
    6: (0, 1, 1),
    7: (1, 0.75, 0.8),
    8: (0.8, 1, 0),
    9: (.588, .294, 0.)
}

r = (1 - 0.1913)
color_values = np.array(list(COLOR_MAP.values()))
m_rgb = color_values.mean(axis=0)
std_rgb = color_values.std(axis=0)

bmnist_mean = [r*m + MNIST_MEAN[0] for m in m_rgb]
bmnist_std = [(r*s**2+(1-r)*MNIST_STD[0]**2+r*(1-r)*(bmnist_mean[i] - m_rgb[i])**2)**0.5 for i, s in enumerate(std_rgb)]

unnormalize = torchvision.transforms.Normalize([-m/s for m, s in zip(bmnist_mean, bmnist_std)], [1/s for s in bmnist_std])
sample_idx = 95
to_pil_image(unnormalize(benchmark.trains[2][sample_idx][0]), mode="RGB")

In [5]:
target_label = 0
incremental_step = 1
# cat_img = torch.cat([img for img in benchmark.trains[incremental_step].inputs[benchmark.trains[incremental_step].targets == target_label][20:30]], dim=2)
cat_img = torch.cat([img for img, target, *_ in benchmark.tests[incremental_step]][20:30], dim=2)
to_pil_image(unnormalize(cat_img), mode="RGB")

In [6]:
from trainers import FairContinualTrainer
from trainers.fair_trainer import FairContinualTrainer1
from metrics import FairMetricCollector
from algorithms.fairl import FaIRL
from algorithms.fairl import FaIRL
from backbones import MLP2Layers2, ResNet18Small2

# backbone = MLP2Layers2(input_dim=n_feature, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
backbone = ResNet18Small2(
        input_dim=input_dim, 
        output_dim=num_classes,
        class_idx=class_idx,
        config=params
    ).to(params['device'])
algorithm = FaIRL(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

trainer = FairContinualTrainer1(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 73.5713698592604, 'loss': 0.0032524381910342417, 'multiclass_eo': [0.42118550814907085, 0.6459474701901385], 'accuracy_s0': 99.25482894647351, 'accuracy_s1': 45.898180029513036, 'classwise_accuracy': {4: array([780, 982]), 5: array([604, 892])}}
[2] Eval metrics for task 1 >> {'accuracy': 74.6765774981049, 'loss': 0.0024691499539093377, 'multiclass_eo': [0.39171182393854453, 0.6430671930389945], 'accuracy_s0': 99.57810480854248, 'accuracy_s1': 47.83915395966552, 'classwise_accuracy': {4: array([794, 982]), 5: array([611, 892])}}
[3] Eval metrics for task 1 >> {'accuracy': 74.71208205090736, 'loss': 0.0023083250607853893, 'multiclass_eo': [0.3873684210526316, 0.654568159845311], 'accuracy_s0': 99.78448275862068, 'accuracy_s1': 47.68765371372356, 'classwise_accuracy': {4: array([798, 982]), 5: array([608, 892])}}
[4] Eval metrics for task 1 >> {'accuracy': 75.82242700446604, 'loss': 0.

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[89.873,  0.   ,  0.   ,  0.   ,  0.   ],
       [83.669, 49.881,  0.   ,  0.   ,  0.   ],
       [56.745, 59.669, 49.534,  0.   ,  0.   ],
       [51.395, 49.234, 48.427, 12.365,  0.   ],
       [46.27 , 40.235, 40.714, 24.429, 44.758]])

In [9]:
[np.round(x, 3) for x in metric_manager_callback.meters['accuracy'].compute_overall()]

TypeError: 'numpy.float64' object is not iterable

In [10]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

58.31999894627133

In [11]:
metric_manager_callback.meters['accuracy'].compute_final()

39.28119091305616

In [12]:
[np.round(x, 3) for x in metric_manager_callback.meters['multiclass_eo'].compute_overall()]

[0.252, 0.356, 0.572, 0.714, 0.689]

In [13]:
np.mean(metric_manager_callback.meters['multiclass_eo'].compute_overall())

0.5165693908539541

In [14]:
task_id = 2

print(f"{task_id=}")
print(f"sensitive samples / all samples = {(benchmark.trains[task_id].sensitives != benchmark.trains[task_id].targets).sum().item()} / {benchmark.trains[task_id].sensitives.shape[0]}")

updated_seq_indices = benchmark.seq_indices_train[task_id]
print(f"sensitive samples / selected samples = {(benchmark.trains[task_id].sensitives[updated_seq_indices] != benchmark.trains[task_id].targets[updated_seq_indices]).sum().item()} / {len(updated_seq_indices)}")


task_id=2
sensitive samples / all samples = 644 / 12700
sensitive samples / selected samples = 503 / 10000


In [15]:
step_class = 2
for i in range(2, 6):
    incremental_step = i
    print(f"{incremental_step=}")
    one_idx = benchmark.trains[incremental_step].sample_weight > 0.9

    print(f"{2*i-2} : {(benchmark.trains[incremental_step].targets == (2*i-2)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-2)).sum().item()}")
    print(f"{2*i-1} : {(benchmark.trains[incremental_step].targets == (2*i-1)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-1)).sum().item()}")


incremental_step=2
2 : 5958 --> 5958
3 : 0 --> 0
incremental_step=3
4 : 0 --> 0
5 : 0 --> 0
incremental_step=4
6 : 0 --> 0
7 : 6265 --> 6265
incremental_step=5
8 : 5851 --> 5851
9 : 0 --> 0


In [16]:
for eos in metric_manager_callback.meters['multiclass_eo'].get_data():
    for eo in eos:
        print(round(eo, 3),end=" ")
    print()

0.168 0.252 
0.293 0.356 0.304 0.082 
0.42 0.282 0.572 0.331 0.398 0.368 
0.445 0.338 0.312 0.714 0.295 0.617 0.127 0.226 
0.383 0.486 0.242 0.689 0.524 0.467 0.178 0.304 0.636 0.384 


In [17]:
metric_manager_callback.meters['classwise_accuracy'].get_data()

[{4: array([902, 982]), 5: array([784, 892])},
 {4: array([833, 982]),
  5: array([736, 892]),
  1: array([ 523, 1135]),
  2: array([ 554, 1032])},
 {5: array([520, 892]),
  4: array([542, 982]),
  1: array([ 699, 1135]),
  2: array([ 596, 1032]),
  6: array([462, 958]),
  9: array([ 513, 1009])},
 {5: array([440, 892]),
  4: array([525, 982]),
  2: array([ 487, 1032]),
  1: array([ 582, 1135]),
  6: array([437, 958]),
  9: array([ 517, 1009]),
  0: array([127, 980]),
  7: array([ 121, 1028])},
 {5: array([384, 892]),
  4: array([486, 982]),
  2: array([ 374, 1032]),
  1: array([ 502, 1135]),
  9: array([ 394, 1009]),
  6: array([406, 958]),
  0: array([291, 980]),
  7: array([ 197, 1028]),
  3: array([ 480, 1010]),
  8: array([409, 974])}]